## How would we get the code written with rgb if it was done with y channel?

In [19]:
import os
import glob
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers
from math import log10

In [4]:
class SRCNN(tf.keras.Model):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = layers.Conv2D(64, (9, 9), activation='relu', padding='same')
        self.conv2 = layers.Conv2D(32, (1, 1), activation='relu', padding='same')
        self.conv3 = layers.Conv2D(1, (5, 5), activation="linear", padding='same')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return x


In [23]:
class SRCNN_Trainer:
    def __init__(self, train_path, val_path, nImgs, nEpochs, batch_size, print_every, model_path, output_path):
        self.train_path = train_path
        self.val_path = val_path
        self.nImgs = nImgs
        self.nEpochs = nEpochs
        self.batch_size = batch_size
        self.print_every = print_every
        self.model_path = model_path
        self.output_path = output_path
        os.makedirs(self.output_path, exist_ok=True)

        self.data_files = sorted(glob.glob(os.path.join(self.train_path, 'Data', '*.png')))[:nImgs]
        self.label_files = sorted(glob.glob(os.path.join(self.train_path, 'Labels', '*.png')))[:nImgs]

        print(f"Number of data files: {len(self.data_files)}")
        print(f"Number of label files: {len(self.label_files)}")

        self.model = SRCNN()
        self.model(tf.zeros((1, 256, 256, 1)))
        self.model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='mse')
        self.model.summary()

    def load_y_channel(self, file_path):
        try:
            img = Image.open(file_path).convert("YCbCr")
            y, _, _ = img.split()
            y = y.resize((256, 256))
            return np.array(y, dtype=np.float32) / 255.0
        except Exception as e:
            print(f"❌ Hata: {file_path} - {e}")
            return None

    def read_batch(self):
        idx = np.random.choice(len(self.data_files), self.batch_size, replace=False)
        imgs, labels = [], []
        for i in idx:
            inp = self.load_y_channel(self.data_files[i])
            lbl = self.load_y_channel(self.label_files[i])
            if inp is not None and lbl is not None:
                imgs.append(np.expand_dims(inp, axis=-1))
                labels.append(np.expand_dims(lbl, axis=-1))
        return np.array(imgs), np.array(labels)

    def train(self):
        print(">>> Eğitim başlatıldı.")
        for epoch in range(self.nEpochs):
            print(f"\nEpoch {epoch+1}/{self.nEpochs}")
            for i in range(len(self.data_files) // self.batch_size):
                x, y = self.read_batch()
                loss = self.model.train_on_batch(x, y)
                if (i + 1) % self.print_every == 0:
                    print(f"[{i+1}] Loss: {loss:.4f}")

        self.model.save_weights(self.model_path)
        print(f"\nModel kaydedildi: {self.model_path}")
        self.predict()


    def calculate_psnr(self, pred_img, label_img):
        """PSNR hesaplama fonksiyonu."""
        mse = np.mean((pred_img.astype(np.float32) - label_img.astype(np.float32)) ** 2)
        psnr = 100 if mse == 0 else 10 * log10((255 ** 2) / mse)
        return psnr

    def predict(self):
        val_files = sorted(glob.glob(os.path.join(self.val_path, "*.png")))
        print(f"{len(val_files)} validasyon görseli bulundu.")
    
        self.model.load_weights(self.model_path)
    
        psnr_total = 0.0
        count = 0
    
        # ✅ Label dizini tanımı
        label_dir = os.path.join(os.path.dirname(self.val_path), "Labels")
    
        for i, file in enumerate(val_files):
            try:
                base_name = os.path.basename(file)
    
                img = Image.open(file).convert("YCbCr")
                y, cb, cr = img.split()
    
                y_input = np.array(y, dtype=np.float32) / 255.0
                y_input = np.expand_dims(np.expand_dims(y_input, axis=0), axis=-1)
    
                pred = self.model.predict(y_input)[0, :, :, 0]
                pred = np.clip(pred * 255.0, 0, 255).astype(np.uint8)
    
                # ✅ Doğru label yolu oluştur
                label_path = os.path.join(label_dir, base_name)
                if not os.path.exists(label_path):
                    print(f"🚫 Eşleşen label yok: {label_path}")
                    continue
    
                label_img = Image.open(label_path).convert("YCbCr")
                label_y, _, _ = label_img.split()
                label_y = np.array(label_y, dtype=np.uint8)
    
                psnr = self.calculate_psnr(pred, label_y)
                psnr_total += psnr
                count += 1
                print(f"✔ PSNR: {psnr:.2f} dB - {base_name}")
    
                # ✅ RGB olarak kaydet
                pred_y = Image.fromarray(pred)
                out_img = Image.merge("YCbCr", [pred_y, cb, cr]).convert("RGB")
                save_path = os.path.join(self.output_path, f"pred_{base_name}")
                out_img.save(save_path)
                print(f"{i+1}. çıktı kaydedildi: {save_path}")
    
            except Exception as e:
                print(f"❌ Hata oluştu: {file} - {e}")
    
        if count > 0:
            avg_psnr = psnr_total / count
            print(f"\n📊 Ortalama PSNR: {avg_psnr:.2f} dB ({count} görsel)")
        else:
            print("🚫 PSNR hesaplanamadı, label dosyaları eksik olabilir.")


if __name__ == "__main__":
    trainer = SRCNN_Trainer(
        train_path=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Dataset\Train",
        val_path=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Dataset\Val\Data",
        nImgs=684,
        nEpochs=10,
        batch_size=16,
        print_every=10,
        model_path=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Super-Resolution-master\trained_SRCNN_model_y.weights.h5",
        output_path=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Super-Resolution-master\Predictions_Y"
    )
    trainer.train()  

Number of data files: 684
Number of label files: 684


Model: "srcnn_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)                   │ (1, 256, 256, 64)           │           5,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (1, 256, 256, 32)           │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (1, 256, 256, 1)            │             801 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,129 (31.75 KB)

 Trainable params: 8,129 (31.75 KB)

 Non-trainable params: 0 (0.00 B)

>>> Eğitim başlatıldı.

Epoch 1/10
[10] Loss: 0.2685
[20] Loss: 0.2201
[30] Loss: 0.1783
[40] Loss: 0.1414

Epoch 2/10
[10] Loss: 0.1111
[20] Loss: 0.0947
[30] Loss: 0.0826
[40] Loss: 0.0735

Epoch 3/10
[10] Loss: 0.0651
[20] Loss: 0.0595
[30] Loss: 0.0548
[40] Loss: 0.0509

Epoch 4/10
[10] Loss: 0.0469
[20] Loss: 0.0440
[30] Loss: 0.0415
[40] Loss: 0.0393

Epoch 5/10
[10] Loss: 0.0369
[20] Loss: 0.0352
[30] Loss: 0.0336
[40] Loss: 0.0321

Epoch 6/10
[10] Loss: 0.0306
[20] Loss: 0.0294
[30] Loss: 0.0283
[40] Loss: 0.0273

Epoch 7/10
[10] Loss: 0.0262
[20] Loss: 0.0254
[30] Loss: 0.0246
[40] Loss: 0.0238

Epoch 8/10
[10] Loss: 0.0230
[20] Loss: 0.0223
[30] Loss: 0.0217
[40] Loss: 0.0211

Epoch 9/10
[10] Loss: 0.0205
[20] Loss: 0.0200
[30] Loss: 0.0195
[40] Loss: 0.0190

Epoch 10/10
[10] Loss: 0.0185
[20] Loss: 0.0181
[30] Loss: 0.0177
[40] Loss: 0.0173

Model kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Super-Resolution-master\trained_SRCNN_model_y.weights.h5
17